# DistilBERT 사전 학습

1. 말뭉치 준비
2. 토크나이저 훈련
3. 데이터셋 준비
4. 모델 구성 및 훈련
    1. 작은 모델
    2. 기반 모델

## 1. 말뭉치 준비

In [1]:
corpus_file = "won/data/won04-gyojeon.txt"

In [2]:
sample_text_for_test = "물질이 개벽되니 정신을 개벽하자"

sample_texts_for_test = [
    "물질이 개벽되니 정신을 개벽하자",
    "19.대종사 말씀하시기를 [스승이 법을 새로 내는 일이나"
]

masked_texts_for_test = [
    "물질이 개벽되니 [MASK]을 개벽하자",
    "19.대종사 말씀하시기를 [스승이 [MASK]을 새로 내는 일이나",
    "19.대종사 [MASK]기를"
]

## 2. 토크나이저 훈련

허깅 페이스 [Tokenizers](https://huggingface.co/docs/tokenizers/index) 라이브러리를 사용하여 토크나이저를 훈련시키고 결과를 저장합니다. 알고리즘과 단어 개수를 변경해 가면서 각각의 결과를 개별 디렉토리에 저장합니다.

* 알고리즘: BPE, Unigram, WordPiece
* 단어 개수: 2000, 3000, 4000, 5000

In [3]:
import os
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram, WordPiece
from tokenizers.trainers import BpeTrainer, UnigramTrainer, WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace

def create_tokenizer_and_trainer(model_name, vocab_size):
    if model_name == "bpe":
        tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
        trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                            vocab_size=vocab_size,
                            limit_alphabet=limit_alphabet,
                            min_frequency=min_frequency,
                            show_progress=False)
    elif model_name == "unigram":
        tokenizer = Tokenizer(Unigram())
        trainer = UnigramTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                            unk_token="[UNK]",
                            vocab_size=vocab_size,
                            show_progress=False)
    elif model_name == "wordpiece":
        tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
        trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                            vocab_size=vocab_size,
                            limit_alphabet=limit_alphabet,
                            min_frequency=min_frequency,
                            show_progress=False)

    tokenizer.pre_tokenizer = Whitespace()

    return tokenizer, trainer

data_files = [
    corpus_file
]

output_dir = "won/tokenizers"
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)

model_names = ["bpe", "unigram", "wordpiece"]
vocab_sizes = [2000, 3000, 4000, 5000]
limit_alphabet = 6000
min_frequency = 5

for model_name in model_names:
    for vocab_size in vocab_sizes:
        # 토크나이저와 트레이너 생성
        tokenizer, trainer = create_tokenizer_and_trainer(model_name, vocab_size)
        # 훈련
        tokenizer.train(data_files, trainer)
        # 저장
        tokenizer_dir = os.path.join(output_dir, f"{model_name}_{vocab_size}")
        if not os.path.exists(tokenizer_dir):
            os.makedirs(tokenizer_dir, exist_ok=True)
        tokenizer_file = os.path.join(tokenizer_dir, "tokenizer.json")
        tokenizer.save(tokenizer_file)
        print(f"Tokenizer file: {tokenizer_file}")

Tokenizer file: won/tokenizers/bpe_2000/tokenizer.json
Tokenizer file: won/tokenizers/bpe_3000/tokenizer.json
Tokenizer file: won/tokenizers/bpe_4000/tokenizer.json
Tokenizer file: won/tokenizers/bpe_5000/tokenizer.json
Tokenizer file: won/tokenizers/unigram_2000/tokenizer.json
Tokenizer file: won/tokenizers/unigram_3000/tokenizer.json
Tokenizer file: won/tokenizers/unigram_4000/tokenizer.json
Tokenizer file: won/tokenizers/unigram_5000/tokenizer.json
Tokenizer file: won/tokenizers/wordpiece_2000/tokenizer.json
Tokenizer file: won/tokenizers/wordpiece_3000/tokenizer.json
Tokenizer file: won/tokenizers/wordpiece_4000/tokenizer.json
Tokenizer file: won/tokenizers/wordpiece_5000/tokenizer.json


토크나이저 파일을 읽어서 토크나이저를 생성하고 예시 문장에 적용해 봅니다.

In [4]:
import sys, traceback
from transformers import DistilBertTokenizerFast, PreTrainedTokenizerFast

def tokenize_and_print(model_names, vocab_sizes, texts):
    for text in texts:
        print("#" * 80)
        print("TEXT: '" + text + "'")
        for model_name in model_names:
            print(f">>> {model_name}")
            for vocab_size in vocab_sizes:
                try:
                    tokenizer_dir = os.path.join(output_dir, f"{model_name}_{vocab_size}")
                    tokenizer_file = os.path.join(output_dir, f"{model_name}_{vocab_size}", "tokenizer.json")
                    if True:
                        # Tokenizers 라이브러리 사용
                        tokenizer = Tokenizer.from_file(tokenizer_file)
                        print(f"{vocab_size:5}: {tokenizer.encode(text).tokens}")
                    else:
                        # Transformers 라이브러리 사용
                        #tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_dir, # Success
                        #tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file, # Success
                        tokenizer = DistilBertTokenizerFast(tokenizer_file=tokenizer_file, # Error
                            unk_token="[UNK]",
                            cls_token="[CLS]",
                            sep_token="[SEP]",
                            pad_token="[PAD]",
                            mask_token="[MASK]")
                        print(f"{vocab_size:5}: {tokenizer.tokenize(text)}")
                except Exception as e:
                    print(f"{vocab_size:5}: FAIL - {e}")
                    traceback.print_exc(file=sys.stdout)

tokenize_and_print(model_names, vocab_sizes, sample_texts_for_test)

################################################################################
TEXT: '물질이 개벽되니 정신을 개벽하자'
>>> bpe
 2000: ['물', '질', '이', '개', '벽', '되', '니', '정신', '을', '개', '벽', '하', '자']
 3000: ['물질', '이', '개', '벽', '되', '니', '정신을', '개', '벽', '하', '자']
 4000: ['물질', '이', '개', '벽', '되니', '정신을', '개', '벽', '하', '자']
 5000: ['물질', '이', '개벽', '되니', '정신을', '개벽', '하자']
>>> unigram
 2000: ['물질', '이', '개', '벽', '되', '니', '정신', '을', '개', '벽', '하자']
 3000: ['물질', '이', '개', '벽', '되', '니', '정신', '을', '개', '벽', '하자']
 4000: ['물질', '이', '개벽', '되', '니', '정신', '을', '개벽', '하자']
 5000: ['물질', '이', '개벽', '되니', '정신', '을', '개벽', '하자']
>>> wordpiece
 2000: ['물', '##질', '##이', '개', '##벽', '##되', '##니', '정', '##신', '##을', '개', '##벽', '##하', '##자']
 3000: ['물', '##질', '##이', '개', '##벽', '##되', '##니', '정', '##신', '##을', '개', '##벽', '##하', '##자']
 4000: ['물질', '##이', '개', '##벽', '##되', '##니', '정신을', '개', '##벽', '##하', '##자']
 5000: ['물질', '##이', '개', '##벽', '##되', '##니', '정신을', '개', '##벽', '##하', '##자']
#######

## 3. 데이터셋 준비

In [5]:
tokenizer_dir = os.path.join(output_dir, "unigram_4000")
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    tokenizer_dir,
    unk_token="[UNK]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    pad_token="[PAD]",
    mask_token="[MASK]"
)
print(tokenizer.vocab_size)

4000


In [6]:
# tokenizer()
inputs = tokenizer(sample_text_for_test)
print(inputs)

# tokenizer.tokenize()
tokenized_inputs = tokenizer.tokenize(sample_text_for_test)
print(tokenized_inputs)

# tokenizer.encode()
encoded_ids = tokenizer.encode(sample_text_for_test)
encoded_tokens =tokenizer.convert_ids_to_tokens(encoded_ids)
print(encoded_tokens)

{'input_ids': [315, 7, 2390, 59, 60, 161, 5, 2390, 326], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
['물질', '이', '개벽', '되', '니', '정신', '을', '개벽', '하자']
['물질', '이', '개벽', '되', '니', '정신', '을', '개벽', '하자']


In [7]:
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling

# MLM을 위한 데이터셋
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=corpus_file,
    block_size=512  # 토큰 기준 최대 길이
)
print(len(dataset))
print(tokenizer.decode(dataset[-1]["input_ids"]))

# MLM 데이터 콜레이터
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

1082
19 . 대종사 말씀하시 기 를 [ 스승이 법 을 새로 내 는 일이 나 , 제자들 이 그 법 을 받아 서 후 래 대중에게 전 하는 일이 나 , 또 후 래 대중 이 그 법 을 반 가 이 받들어 실행 하는 일이 삼 위 일체 ( 三 位 一 體 ) 되는 일이라 , 그 공덕 도 또 한 다름 이 없나니 라 . ]


/home/wjeong/DevEnv/py39/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


## 4. 모델 구성 및 훈련

### A. 작은 모델

In [8]:
from transformers import DistilBertConfig, DistilBertForMaskedLM

def create_model_for_mlm(
    vocab_size,
    dim, 
    hidden_dim,
    n_layers,
    n_heads,
    max_position_embeddings,
    output_attentions=False
):
    # 표준 DistilBERT-base 구성과 유사하게 설정
    config = DistilBertConfig(
        vocab_size=vocab_size, # 로드한 토크나이저의 어휘 크기 사용
        activation="gelu",
        dim=dim,
        hidden_dim=hidden_dim,
        n_layers=n_layers,
        n_heads=n_heads,
        max_position_embeddings=max_position_embeddings,
        output_attentions=output_attentions
        # dropout, attention_dropout 등 다른 파라미터도 설정 가능
    )
    
    # 모델 초기화 (랜덤 가중치)
    model = DistilBertForMaskedLM(config=config)
    print(f"Num of model parameters: {model.num_parameters()}")

    return model

dim = 60                  # Hidden size
hidden_dim = 240          # Intermediate size (dim * 4)
n_layers = 3              # Number of layers
n_heads = 6               # Number of attention heads
max_position_embeddings = 512

model = create_model_for_mlm(
    vocab_size=tokenizer.vocab_size,
    dim=dim,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    n_heads=n_heads,
    max_position_embeddings=max_position_embeddings,
    output_attentions=True
)

Num of model parameters: 410560


[BertViz](https://github.com/jessevig/bertviz)를 사용하여 모델 어텐션 값을 시각화합니다.

In [9]:
from bertviz import model_view

def get_input_tokens_and_attentions(tokenizer, model, input_text):
    tokenized_data = tokenizer(input_text)
    print(tokenized_data)
    tokenized_text = tokenizer.tokenize(input_text)
    print(tokenized_text)
    encoded_inputs = tokenizer.encode(input_text, return_tensors='pt')  # Tokenize input text
    print(encoded_inputs)

    outputs = model(encoded_inputs)  # Run model
    print(outputs.keys())
    print(len(outputs.attentions))
    print(type(outputs.attentions[0]))
    print(outputs.attentions[0].shape)
    print(">>>> last layer, 1st sample, 1st head <<<<")
    print(outputs.attentions[-1][0][0])

    input_tokens = tokenizer.convert_ids_to_tokens(encoded_inputs[0])  # Convert input ids to token strings
    attentions = outputs[-1]  # Retrieve attention from model outputs
    
    return input_tokens, attentions

In [10]:
input_tokens, attentions = get_input_tokens_and_attentions(tokenizer, model, sample_text_for_test)
print(input_tokens)
model_view(attentions, input_tokens)  # Display model view

DistilBertSdpaAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


{'input_ids': [315, 7, 2390, 59, 60, 161, 5, 2390, 326], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
['물질', '이', '개벽', '되', '니', '정신', '을', '개벽', '하자']
tensor([[ 315,    7, 2390,   59,   60,  161,    5, 2390,  326]])
odict_keys(['logits', 'attentions'])
3
<class 'torch.Tensor'>
torch.Size([1, 6, 9, 9])
>>>> last layer, 1st sample, 1st head <<<<
tensor([[0.1233, 0.1233, 0.1228, 0.1245, 0.1235, 0.1228, 0.1243, 0.1234, 0.1232],
        [0.1255, 0.1221, 0.1281, 0.1211, 0.0000, 0.1193, 0.0000, 0.1276, 0.1213],
        [0.1267, 0.0000, 0.1239, 0.1247, 0.1223, 0.0000, 0.0000, 0.1263, 0.1253],
        [0.1235, 0.1264, 0.1266, 0.1186, 0.1233, 0.1244, 0.1200, 0.1233, 0.1250],
        [0.1247, 0.1217, 0.1201, 0.1281, 0.0000, 0.1225, 0.1246, 0.0000, 0.1205],
        [0.1210, 0.1214, 0.1255, 0.1242, 0.1215, 0.1259, 0.1217, 0.1244, 0.1256],
        [0.0000, 0.1250, 0.1185, 0.1243, 0.1188, 0.1316, 0.0000, 0.1240, 0.1282],
        [0.1270, 0.1193, 0.12

<IPython.core.display.Javascript object>

In [11]:
from transformers import TrainingArguments, Trainer

os.environ["WANDB_DISABLED"] = "true"

def create_model_trainer(output_dir, n_epochs, batch_size, logging_strategy="steps", logging_steps=500):
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=n_epochs,       # 실제로는 훨씬 더 많이 필요
        per_device_train_batch_size=batch_size, # GPU 메모리에 맞춰 조절
        logging_strategy=logging_strategy,
        logging_first_step=True,
        logging_steps=logging_steps,
        save_steps=10_000,
        save_total_limit=2,
        prediction_loss_only=True,
        fp16=True, # 가능하면 True
        # learning_rate, weight_decay 등 추가 설정 필요
    )
    
    # Trainer 초기화
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    return trainer

# 모델 학습
output_dir = "./won/models/distilbert_pretrained_mlm_small"

n_epochs = 10
batch_size = 8

trainer = create_model_trainer(output_dir, n_epochs, batch_size)
print("DistilBERT 구조 모델 사전 학습(MLM)을 시작합니다. (매우 간소화된 모델)")
trainer.train()
print("사전 학습 완료.")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


DistilBERT 구조 모델 사전 학습(MLM)을 시작합니다. (매우 간소화된 예시)


Step,Training Loss
500,7.846200
1000,7.235800


사전 학습 완료.


In [12]:
# 모델 저장
pretrained_dir = os.path.join(output_dir, "pretrained")
print(f"학습된 모델과 토크나이저를 {pretrained_dir}에 저장합니다.")
trainer.save_model(pretrained_dir) # 모델과 토크나이저 모두 저장됨
#tokenizer.save_pretrained(pretrained_dir) # 사용한 토크나이저 함께 저장
print("저장 완료.")

학습된 모델과 토크나이저를 ./output/distilbert_pretrained_mlm_small/pretrained에 저장합니다.
저장 완료.


In [13]:
# 사용 예시 (저장된 모델 로드)
from transformers import AutoTokenizer, DistilBertForMaskedLM

tokenizer = AutoTokenizer.from_pretrained(pretrained_dir)
model = DistilBertForMaskedLM.from_pretrained(pretrained_dir, output_attentions=True)

print(len(tokenizer))
tokenized_inputs = tokenizer.tokenize(sample_text_for_test)
print(tokenized_inputs)

4000
['물질', '이', '개벽', '되', '니', '정신', '을', '개벽', '하자']


In [14]:
import torch
from datasets import Dataset

#device = "cpu"

def find_topk_for_masked(tokenizer, model, text, topk=5):
    inputs = tokenizer(text, return_tensors="pt")
    #inputs = {k: v.to(device) for k, v in inputs.items() if isinstance(v, torch.Tensor)}
    if 'token_type_ids' in inputs:
        inputs.pop('token_type_ids')

    token_logits = model(**inputs).logits
    #print(token_logits.shape)

    # [MASK]의 위치를 찾고, 해당 logits을 추출합니다.
    #print(torch.where(inputs["input_ids"] == tokenizer.mask_token_id))
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
    #print(mask_token_index)
    mask_token_logits = token_logits[0, mask_token_index, :]
    #print(mask_token_logits)

    # 가장 큰 logits값을 가지는 [MASK] 후보를 선택합니다.
    top_5_tokens = torch.topk(mask_token_logits, topk, dim=1).indices[0].tolist()

    return top_5_tokens

In [15]:
def test_model_for_mlm(tokenizer, model, texts):
    for text in texts:
        print("#" * 80)
        print(f"INPUT    : {text}")
        print("-" * 80)
        try:
            topk_tokens = find_topk_for_masked(tokenizer, model, text, topk=5)
            for token in topk_tokens:
                print(f"PREDICTED: {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")
        except Exception as e:
            print(f"Exception: {e}")

test_model_for_mlm(tokenizer, model, masked_texts_for_test)

################################################################################
INPUT    : 물질이 개벽되니 [MASK]을 개벽하자
--------------------------------------------------------------------------------
PREDICTED: 물질이 개벽되니 을을 개벽하자
PREDICTED: 물질이 개벽되니 ,을 개벽하자
PREDICTED: 물질이 개벽되니 를을 개벽하자
PREDICTED: 물질이 개벽되니 이을 개벽하자
PREDICTED: 물질이 개벽되니 .을 개벽하자
################################################################################
INPUT    : 19.대종사 말씀하시기를 [스승이 [MASK]을 새로 내는 일이나
--------------------------------------------------------------------------------
PREDICTED: 19.대종사 말씀하시기를 [스승이 ,을 새로 내는 일이나
PREDICTED: 19.대종사 말씀하시기를 [스승이 을을 새로 내는 일이나
PREDICTED: 19.대종사 말씀하시기를 [스승이 를을 새로 내는 일이나
PREDICTED: 19.대종사 말씀하시기를 [스승이 .을 새로 내는 일이나
PREDICTED: 19.대종사 말씀하시기를 [스승이 이을 새로 내는 일이나
################################################################################
INPUT    : 19.대종사 [MASK]기를
--------------------------------------------------------------------------------
PREDICTED: 19.대종사 ,기를
PREDICTED: 19.대종사 을기를
PREDICT

In [16]:
input_tokens, attentions = get_input_tokens_and_attentions(tokenizer, model, sample_text_for_test)
print(input_tokens)
model_view(attentions, input_tokens)  # Display model view

{'input_ids': [315, 7, 2390, 59, 60, 161, 5, 2390, 326], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
['물질', '이', '개벽', '되', '니', '정신', '을', '개벽', '하자']
tensor([[ 315,    7, 2390,   59,   60,  161,    5, 2390,  326]])
odict_keys(['logits', 'attentions'])
3
<class 'torch.Tensor'>
torch.Size([1, 6, 9, 9])
>>>> last layer, 1st sample, 1st head <<<<
tensor([[0.0809, 0.1238, 0.0971, 0.1164, 0.1255, 0.1097, 0.1183, 0.1043, 0.1239],
        [0.0621, 0.1360, 0.0869, 0.1137, 0.1372, 0.1090, 0.1226, 0.0989, 0.1335],
        [0.0684, 0.1317, 0.0903, 0.1153, 0.1337, 0.1092, 0.1214, 0.1001, 0.1299],
        [0.0702, 0.1312, 0.0923, 0.1125, 0.1317, 0.1106, 0.1201, 0.1013, 0.1300],
        [0.0592, 0.1380, 0.0835, 0.1153, 0.1405, 0.1090, 0.1243, 0.0963, 0.1338],
        [0.0619, 0.1346, 0.0883, 0.1152, 0.1360, 0.1093, 0.1224, 0.0985, 0.1339],
        [0.0656, 0.1324, 0.0893, 0.1150, 0.1326, 0.1105, 0.1201, 0.1007, 0.1338],
        [0.0657, 0.1330, 0.08

<IPython.core.display.Javascript object>

### B. 기반 모델

In [17]:
from transformers import DistilBertConfig, DistilBertForMaskedLM

dim = 768                 # Hidden size
hidden_dim = 3072         # Intermediate size (dim * 4)
n_layers = 6              # Number of layers
n_heads = 12              # Number of attention heads
max_position_embeddings = 512

### For TEST ###
#dim = 60                  # Hidden size
#hidden_dim = 240          # Intermediate size (dim * 4)
#n_layers = 3              # Number of layers
#n_heads = 6               # Number of attention heads
#max_position_embeddings = 512

model = create_model_for_mlm(
    vocab_size=tokenizer.vocab_size,
    dim=dim,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    n_heads=n_heads,
    max_position_embeddings=max_position_embeddings,
    output_attentions=True
)

Num of model parameters: 46590112


In [18]:
# 모델 학습
output_dir = "./won/models/distilbert_pretrained_mlm_base"

n_epochs = 800
batch_size = 16

## For TEST
#n_epochs = 10
#batch_size = 16

trainer = create_model_trainer(output_dir, n_epochs, batch_size, logging_steps=5000)
print("DistilBERT 구조 모델 사전 학습(MLM)을 시작합니다. (기반 모델)")
trainer.train()
print("사전 학습 완료.")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


DistilBERT 구조 모델 사전 학습(MLM)을 시작합니다. (기반 예시)


Step,Training Loss
5000,5.252500
10000,3.484700
15000,1.619700
20000,0.530300
25000,0.203300
30000,0.109500
35000,0.068600
40000,0.046800
45000,0.033500
50000,0.024500


사전 학습 완료.


In [19]:
# 모델 저장
pretrained_dir = os.path.join(output_dir, "pretrained")
print(f"학습된 모델과 토크나이저를 {pretrained_dir}에 저장합니다.")
trainer.save_model(pretrained_dir) # 모델과 토크나이저 모두 저장됨
#tokenizer.save_pretrained(pretrained_dir) # 사용한 토크나이저 함께 저장
print("저장 완료.")

학습된 모델과 토크나이저를 ./output/distilbert_pretrained_mlm_base/pretrained에 저장합니다.
저장 완료.


In [20]:
# 사용 예시 (저장된 모델 로드)
from transformers import AutoTokenizer, DistilBertForMaskedLM

tokenizer = AutoTokenizer.from_pretrained(pretrained_dir)
model = DistilBertForMaskedLM.from_pretrained(pretrained_dir, output_attentions=True)

print(len(tokenizer))
tokenized_inputs = tokenizer.tokenize(sample_text_for_test)
print(tokenized_inputs)

4000
['물질', '이', '개벽', '되', '니', '정신', '을', '개벽', '하자']


In [21]:
test_model_for_mlm(tokenizer, model, masked_texts_for_test)

################################################################################
INPUT    : 물질이 개벽되니 [MASK]을 개벽하자
--------------------------------------------------------------------------------
PREDICTED: 물질이 개벽되니 자력을 개벽하자
PREDICTED: 물질이 개벽되니 물질을 개벽하자
PREDICTED: 물질이 개벽되니 정신을 개벽하자
PREDICTED: 물질이 개벽되니 )을 개벽하자
PREDICTED: 물질이 개벽되니 힘을 개벽하자
################################################################################
INPUT    : 19.대종사 말씀하시기를 [스승이 [MASK]을 새로 내는 일이나
--------------------------------------------------------------------------------
PREDICTED: 19.대종사 말씀하시기를 [스승이 법을 새로 내는 일이나
PREDICTED: 19.대종사 말씀하시기를 [스승이 대중을 새로 내는 일이나
PREDICTED: 19.대종사 말씀하시기를 [스승이 이익을 새로 내는 일이나
PREDICTED: 19.대종사 말씀하시기를 [스승이 마음을 새로 내는 일이나
PREDICTED: 19.대종사 말씀하시기를 [스승이 공부인을 새로 내는 일이나
################################################################################
INPUT    : 19.대종사 [MASK]기를
--------------------------------------------------------------------------------
PREDICTED: 19.대종사 말씀하시기를
PREDICTED: 19.대종사 

In [22]:
input_tokens, attentions = get_input_tokens_and_attentions(tokenizer, model, sample_text_for_test)
print(input_tokens)
model_view(attentions, input_tokens)  # Display model view

{'input_ids': [315, 7, 2390, 59, 60, 161, 5, 2390, 326], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
['물질', '이', '개벽', '되', '니', '정신', '을', '개벽', '하자']
tensor([[ 315,    7, 2390,   59,   60,  161,    5, 2390,  326]])
odict_keys(['logits', 'attentions'])
6
<class 'torch.Tensor'>
torch.Size([1, 12, 9, 9])
>>>> last layer, 1st sample, 1st head <<<<
tensor([[0.1704, 0.1285, 0.1901, 0.1140, 0.0854, 0.1212, 0.0288, 0.0927, 0.0688],
        [0.1274, 0.1436, 0.0918, 0.1395, 0.1780, 0.0840, 0.0759, 0.0850, 0.0749],
        [0.1488, 0.0964, 0.1818, 0.1061, 0.0826, 0.1419, 0.0501, 0.1227, 0.0696],
        [0.1336, 0.1432, 0.2261, 0.0910, 0.0714, 0.1389, 0.0568, 0.0941, 0.0448],
        [0.1000, 0.0898, 0.1955, 0.0912, 0.1121, 0.1855, 0.0779, 0.0751, 0.0729],
        [0.1381, 0.1545, 0.0961, 0.1035, 0.1347, 0.1598, 0.0947, 0.0593, 0.0593],
        [0.0948, 0.0769, 0.2240, 0.0705, 0.0657, 0.1771, 0.1256, 0.1085, 0.0568],
        [0.0906, 0.0987, 0.1

<IPython.core.display.Javascript object>